# Prerequisite

In [1]:
# import initial libraries i.e. pandas and numpy
import pandas as pd
import numpy as np

# Load and preview the megaline carrier dataset

In [2]:
# Import megaline subscriber behaviour dataset and preview the first few records
mega_df = pd.read_csv("https://bit.ly/UsersBehaviourTelco")
mega_df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


# Data Exploration

In [5]:
# check the structure of the dataframe
mega_df.shape

(3214, 5)

In [4]:
# check the column datatypes
mega_df.dtypes

calls       float64
minutes     float64
messages    float64
mb_used     float64
is_ultra      int64
dtype: object

In [3]:
# check the mb_used column
mega_df['mb_used'].describe()


count     3214.000000
mean     17207.673836
std       7570.968246
min          0.000000
25%      12491.902500
50%      16943.235000
75%      21424.700000
max      49745.730000
Name: mb_used, dtype: float64